Рекомендательные системмы являются одними из самых популярных приложений DS. Они используются для прогнозирования предпочтений,  которые пользователь поставит какому-нибудь товару в магазине. Amazon использует их, чтобы предлагать продукты клиентам, YouTube решает какое видео воспроизводить следующим, а Facebook рекомендует посты. Существуют также рекомендательные системы для таких доменов, как рестораны, фильмы и онлайн-знакомства. Более того, для некоторых компаний, таких как Netflix, бизнес-модель и ее успех зависят от эффективности их рекомендаций. Netflix даже предложил миллион долларов в 2009 году каждому, кто сможет улучшить его систему рекомендаций на 10%.

Вообще говоря, рекомендательные системы могут быть поделены на 3 типа:
* Базовый подход: предлагайте общие рекомендации каждому пользователю в зависимости от популярности фильма и / или жанра. Например, Кинопоиск 250.
* Content-based рекомендательные системы. Эта система использует метаданные элемента, например жанр, режиссер, описание, актеры и т.д. для фильмов. Общая идея, лежащая в основе этих рекомендательных систем, заключается в том, что если человеку нравится конкретный предмет, ему также понравится предмет, похожий на него. Алгоритм будет использовать метаданные прошлых элементов пользователя. Хорошим примером может служить YouTube, где на основе вашей истории он предлагает вам новые видео, которые вы потенциально можете посмотреть.
* Коллаборативная фильтрация. Эти системы пытаются предсказать рейтинг, который пользователь поставит элементу, на основе прошлых оценок и предпочтений других пользователей. Коллаборативной фильтрации не требуются метаданные элемента.

In [2]:
# Set the repository root as a working directory 

%cd ..

C:\Users\lucky_man\Documents\mipt_recsys_kantor


In [21]:
config_path = 'params.yaml'

# Look on pipelines config 
with open(config_path) as conf_file:
    config = conf_file.read()
    
print(config)

base:
  project: mipt_recsys
  model:
    models_folder: models
  reports:
    reports_folder: reports
  random_state: 42


data_load:
  metadata_csv: data/external/movies_metadata.csv
  links_csv: data/external/links.csv
  ratings_csv: data/external/ratings_small.csv

train_simple_model:
  model:
    model_name: simple_model.joblib
  tfidf:
    max_features: 40000

preprocess_svd:
  metadata_path: data/processed/movies_metadata.csv
  links_path: data/processed/links.csv
  ratings_path: data/processed/ratings_small.csv

train_and_test_svd:
  test_size: 0.25
  model:
    model_name: svd_model.joblib


# Content-based

Идея content-based подхода заключается в том, чтобы по истории действий пользователя создать для него вектор его предпочтений в пространстве предметов и рекомендовать товары, близкие к этому вектору.

Попробуем рекомендовать фильмы, похожие на конкретный фильм. Для этого вычислим попарные косинусные расстояния для всех фильмов на основе их сюжетных описаний и порекомендуем фильмы на основе этой метрики.

### Train model, based on cosine similarity

In [32]:
%%time
from src.pipelines.train_simple import train_simple_model
train_simple_model(config_path)


Wall time: 4min 1s


### Predict for some title 

In [34]:
%%time
from src.pipelines.predict_simple import predict_simple_model
predict_simple_model(config_path, 'The Dark Knight Rises')

12481                                      The Dark Knight
150                                         Batman Forever
1328                                        Batman Returns
585                                                 Batman
15511                           Batman: Under the Red Hood
21194    Batman Unmasked: The Psychology of the Dark Kn...
9230                    Batman Beyond: Return of the Joker
19792              Batman: The Dark Knight Returns, Part 1
18035                                     Batman: Year One
3095                          Batman: Mask of the Phantasm
Name: title, dtype: object
Wall time: 1min 4s


# Сollaborative filtering

### preprocess data 

In [36]:
%%time
from src.pipelines.preprocess_svd import preprocess_svd
preprocess_svd(config_path)

Wall time: 4.35 s


### train and evaluate svd 

In [37]:
%%time
from src.pipelines.train_and_test_svd import train_and_test_svd
train_and_test_svd(config_path)

RMSE: 0.9001
0.9001454029024527
Wall time: 12.6 s


### predict for some user 

In [38]:
%%time
from src.pipelines.predict_svd import predict_svd
predict_svd(config_path, 2)


                         ratings
movies                          
The Godfather           4.570011
On the Waterfront       4.512116
Chinatown               4.498240
Modern Times            4.477477
The Godfather: Part II  4.472773
Fargo                   4.463801
Cinema Paradiso         4.423796
It Happened One Night   4.420678
The African Queen       4.396103
Good Will Hunting       4.391499
Wall time: 3.37 s
